This code measures the change in luminescence intensty (delta_L) upon plasticity induction every 0.2 um along the dendritic length from the plasticity induced spine at all time points from baseline to post induction.

In [ ]:
import os
import pandas as pd
import numpy as np
from statistics import mean
import random
import re
import matplotlib.pyplot as plt

Address = r"Folder_path_to_Fiji_Macro_Output"+"\\"
list_folders =  os.listdir(Address)
list_folders.sort()
Output_dir = Address
#if not os.path.exists(Output_dir):
#    os.makedirs(Output_dir)
df_output = pd.DataFrame()

for folder in list_folders:
    df_output = pd.DataFrame()
    if "Profile_StimPoint" in folder:
        list_files =  os.listdir(Address+folder)
        list_files.sort()
        StimPoint = folder.split("StimPoint")[1]
        for file in list_files:
            TimePoint = file.split("TimePoint")[1].split(".")[0]
            LF_df = pd.read_csv(Address+folder+"\\"+file).iloc[:,[1,2,4,5]]
            LF_df.insert(0, 'TimePoint', TimePoint)
            LF_df.columns = ["TimePoint","Distance","L", "F","LbyF"]
            LF_df = LF_df.astype({'TimePoint':'int'})
            df_output = pd.concat([df_output,LF_df])

        df_output = df_output.sort_values("TimePoint")    
        #print(df_output.dtypes)
        #print(df_output) 
        output_filename = "AllTimePoints_StimPoint"+ StimPoint
        #print(Address+output_filename+".csv")
        #df_output.to_csv(Address+output_filename+".csv", index=False)
        
        base_df = df_output[df_output['TimePoint'] < 10]
        base_df = base_df.groupby('Distance')
        base_df_mean = base_df[["L", "F", "LbyF"]].mean().reset_index()
        #base_df_mean.insert(0, 'TimePoint', 0)
        base_df_mean.columns = ["Distance","L_BaseMean", "F_BaseMean","LbyF_BaseMean"]
        #print(base_df_mean)
        stim_df = df_output[df_output['TimePoint'] == 10].sort_values("Distance").iloc[:,1:]   
        stim_df.columns = ["Distance","L_Stim", "F_Stim","LbyF_Stim"]
        #print(stim_df)
        delta = pd.merge(base_df_mean, stim_df, how="outer", on=["Distance"])
        delta = delta.assign(delta_LbyF = lambda delta: delta['LbyF_Stim'] - delta['LbyF_BaseMean'])
        delta = delta.assign(delta_L = lambda delta: delta['L_Stim'] - delta['L_BaseMean'])
        delta = delta.assign(delta_F = lambda delta: delta['F_Stim'] - delta['F_BaseMean'])
        #print(delta)
        
        output_filename = "Delta_StimPoint"+ StimPoint
        delta.to_csv(Address+output_filename+".csv", index=False)
        #print(delta["delta_LbyF"])
        fig, ax = plt.subplots()
        ax.plot(delta["Distance"][::],
                delta["delta_L"][::],
                linewidth=1.0, color = "lightgrey")
        ax.plot(delta["Distance"].rolling(10).mean()[::], 
                delta["delta_L"].rolling(10).mean()[::], 
                linewidth=2.0, color = "orange")
        
        plt.title("Δ L StimPoint"+ StimPoint)
        plt.xlabel("Distance from Stimulation")
        plt.ylabel("Δ L")
        
        plt.savefig(Address+output_filename+".png", bbox_inches='tight', dpi = 300)